In [19]:
import scipy
import scipy.optimize
from scipy.optimize import fsolve

from math import pi
from math import sqrt
from math import sin

import numpy as np
import pandas as pd

import pickle
from pyDOE import *

import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display

pd.options.display.float_format = '{:,.2f}'.format

In [20]:
def sizing_code(param, arg):

    #Design variables : REMOVE eD and J IF INDUCTOR IS PART OF THE MOTOR
    # ---
    J_SL = param[0]              #current density [0, 1] 0 the motor is off, 1 full power
    d_over_l_SL = param[1]       #form factor [0.1,1]*2 <=> [0.2,2]
    f = param[2]                 #switching frequency [2, 40] kHz
    eD = param[3]                #airgap/diameter INDUCTOR [10^-3, 10^-1]
    J = param[4]                 #current density [A/mm^2] INDUCTOR [1, 20] A/mm^2
    DH = param[5]                #aspect ratio CAPACITOR [0.3, 0.8]
    kc = param[6]                #oversizing coefficient CAPACITOR [1, 5]
    kIGBT = param[7]             #oversizing coefficient IGBT [0.5, 10]
    Wh = param[8]                #width heatsink [mm] IGBT [40, 120]mm 



    #Propeller parameters
    # ---
    RPM = 2500
    n = RPM/60
    R = 1                              #PROPELLER RADIUS
    D = 2*R
    omega = n*2*np.pi
   # Power = 211084                      #Max power
   # Thrust = 3187.8                     #Max Thrust
   # Torque = Power/(2*np.pi*n)
    
    Torque = 555.25                      # RMS of the torque (Power/(2*pi*n))  From excel file  
    Power = Torque*(2*np.pi*n)           # Power and torque in the cruise phase (most lasting phase) 

    #Reference motor SIEMENS SP-260D
    Torque_ref = 1000                                                                       # (Nm)
    RPM_ref = 3750                                                                          # (Tr/min) Optimum RPM
    Pj_ref = 10000                                                                          # Joule losses (W)
    Pi_ref = 6000                                                                           # Iron losses (W)
    Pc_ref = 16000                                                                          # Total losses (W)
    m_motor_ref = 50                                                                        # Motor mass (kg)
    d_ref = 0.418                                                                           # Diameter (m)
    l_ref = 0.300                                                                           # Length (m)
    Power_ref = 261000                                                                      # Motor power (W)
    f_ref = 750
    
    #Motor Scaling law (_SL)
    # ---
    Power_corr = Power #real amount of power needed taking into accont the motor efficiency
    Torque_SL = Torque/Torque_ref
    Power_SL = Power/Power_ref
    omega_SL = Power_SL/Torque_SL

    omega_ref = 2*np.pi*RPM_ref/60
    Omega = omega_SL*omega_ref

    RPM_new = Omega*60/(2*np.pi)

    #Hypothesis
    B_SL = 1
    m_iron_SL = 1
    d_SL = ((Torque_SL*d_over_l_SL)/(J_SL*B_SL))**(1/3)
    l_SL = d_SL/d_over_l_SL

    #parameters
    m_motor_SL = d_SL*l_SL
    f_SL = omega_SL*d_SL
    Pi_SL = (f_SL**1.5)*m_iron_SL   #FROM IMECHE
    Pj_SL = (J_SL**2)*d_SL*l_SL     #FROM IMECHE
    Pc = Pj_SL*Pj_ref + Pi_SL*Pi_ref
    Pc_SL = Pc/Pc_ref
    kc_SL = Pc_SL/(d_SL*l_SL)
    
    #keep last efficiency value and finding the new one for the loop
    eff_SL = (Power)/(Power+Pc)
    m_motor = m_motor_SL*m_motor_ref
    

    # Gearbox
    # ---
    M0 = RPM_new/RPM                          #optimal motor RPM wrt propeller RPM
    mg = M0
    K = 6.9*10**6                             #coefficient
    Fd2 = 2*Torque/K*(1+1/mg+mg+mg**2)        #weight factor
    C = 2*Torque/K
    #print('for a reduction ratio = ', M0, 'the weigth factor = ', Fd2/C, 'is in line with the graph in the article')
    ksi = 6.1*10**4                           #conversion coefficient
    Af = 0.25                                 #application factor
    m_gear = ksi*Af*Fd2                       #gear mass


    #POWER ELECTRONIC
    # ---
    #Data
    E = 500                                 #bus voltage
    Il = 400                                #motor voltage
    Ul = 400                                #motor current
    

    #Design assumptions
    dE = 0.02*E                             #Max ripple voltage (2%)
    dI = 16.94/100*Il                       #Max ripple current (16.94%)

    #sizing
    # ---
    alpha = Ul/E
    T = 1/f

    #INDUCTOR
    # ---
    #IF THE INDUCTANCE IS PART OF THE MOTOR AND NOT OF THE CONVERTER REMOVE THIS PART
    #L = (1-alpha)*alpha*E*T/dI
    #Il_max = Il+dI/2
    #Il_RMS = Il*np.sqrt(1+(dI/Il)**2/12)
    #dI2 = dI/2

    #projectual hypotesis
    #J_fil = 10.53*10**6
    #eD_n = 0.0999999
    #mu0 = 1.25664*10**(-6)
    #wind_coeff = 0.33
    
    #Em = 0.5*L*Il_max**2             #magnetic energy
    #Pi0 = 3.86*eD_n**(0.344-0.226*np.log10(eD_n)-0.0355*np.log10(eD_n)**2)
    #Dl = (2*Pi0*Em/(mu0*J_fil**2*(0.078+0.195*eD_n)**2*wind_coeff**2))**(1/5)     #inductance diameter
    #Rl = Pi0/(mu0*Dl)                #riluctance
    #N = round(np.sqrt(L*Rl), 0) +1   #number of turns
    #A_fil = Il_max/J_fil             #fil surface
    #A_bob = A_fil*N/wind_coeff
    #B = N*Il_max/Rl/(Dl**2*np.pi*(0.82/2)**2)
    #dB = N*dI2/Rl/(Dl**2*np.pi*(0.82/2)**2)
    #e = eD_n*Dl

    #M1 = wind_coeff*8960*0.153*0.142**3        #0.142 is the dimension A as in picture 
    #M2 = (1-wind_coeff)*1200*0.153*0.142**3
    #M3 = 0.487*7870*0.1472**3
    #M_ind = M1+M2+M3    


    #CAPACITOR
    # ---
    C = alpha*T*(1-alpha)*Il/dE
    Ic_RMS = np.sqrt(alpha*(1-alpha))*Il*np.sqrt(1+1/(12*(1-alpha))*(dI/Il)**2)

    lam = 0.3
    C_n = C*kc

    C_ref = 1000*10**(-6)
    D_ref = 100*10**(-3)
    H_ref = 155*10**(-3)

    Dc = (C_n/C_ref*(DH/(D_ref/H_ref)))**(1/3)*D_ref        #scaling law
    Hc = Dc/DH
    Rc_ref = 3.2*10**(-3)
    Rc = (Dc/D_ref)**(-2)*Rc_ref

    #losses
    tandel = 2*10**(-4)
    Pj_c = Ic_RMS**2*Rc
    P_di = 0.5*C_n*dE**2*2*np.pi*f*tandel

    M_ref = 1.5
    M_c = M_ref*(Dc/D_ref)**2*(Hc/H_ref)


    #CHOPPER
    # ---
    #IGBT
    I_RMS = np.sqrt(alpha)*Il*np.sqrt(1+1/12*(dI/Il)**2)
    I_mean = alpha*Il

    I_n = I_mean*kIGBT
    I_ref = 80
    V_ref = 600
    comm_ref = 8.2*10**(-3)
    R0_ref = 20*10**(-3)
    V0 = 1                                  #voltage drop
    R0 = R0_ref*(I_n/I_ref)**(-1)           #dynamic resistance
    J_comm = comm_ref*(I_n/I_ref)*(E/V_ref)*Il/I_n
    V_max = 900

    #DIODE
    Id_RMS = np.sqrt(1-alpha)*Il*np.sqrt(1+1/12*(dI/Il)**2)
    Id_mean = (1-alpha)*Il
    
    Id_ref = 41
    Vd_ref = 600
    R0d_ref = 15*10**(-3)
    commd_ref = 17.2*10**(-3)
    Id_n = Id_ref*(I_n/I_ref)
    R0d = R0d_ref*(I_n/I_ref)**(-1)
    Jd_comm = commd_ref*(I_n/I_ref)*(E/V_ref)*Il/Id_n

    Lh = 0.26
    ef = Wh*0.025
    N_fin = round((Wh-ef)/(2*ef), 0)
    Md = 2700*Lh*(0.36*Wh**2+0.02*Wh**2*N_fin)

    J_IGBT = V0*I_mean+R0*I_RMS**2+J_comm*f
    J_diode = V0*Id_mean+R0d*Id_RMS**2+Jd_comm*f

    m_power_electronic = M_c+Md


    # ---
    m_tot = m_motor+m_gear+m_power_electronic
    obj = m_tot


    if arg == 'Obj':
        return obj

    elif arg == 'Prt':
    
        #PRINT THE RESULTS IN THE TABLE
        col_names = ['Type', 'Name', 'Value', 'Unit', 'Comment']
        df = pd.DataFrame()
        #df = df.append([{'Type': 'Motor', 'Name': 'Iterations', 'Value': k, 'Unit': '[-]', 'Comment': 'power'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'Propeller power', 'Value': Power, 'Unit': '[W]', 'Comment': 'power'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'Propeller torque', 'Value': Torque, 'Unit': '[Nm]', 'Comment': 'Torque'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'Motor RPM', 'Value': RPM_new, 'Unit': '[tr/min]', 'Comment': 'RPM'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'Magnetic field', 'Value': B_SL, 'Unit': '[Tesla]', 'Comment': 'Magnetic field / Magnetic field ref'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'Current density scaling factor', 'Value': J_SL, 'Unit': '[A/m^2]', 'Comment': 'j / j_ref'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'D/L', 'Value': d_over_l_SL, 'Unit': '[-]', 'Comment': 'Form factor'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'omega_new', 'Value': Omega, 'Unit': '[rad/s]', 'Comment': 'omega new'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'Motor Diameter', 'Value': d_SL*d_ref, 'Unit': '[m]', 'Comment': 'Diameter / Diameter ref'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'Motor Length', 'Value': l_SL*l_ref, 'Unit': '[m]', 'Comment': 'Length / Length ref'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'Frequency', 'Value': f_SL*f_ref, 'Unit': '[Hz]', 'Comment': 'Frequency / Frequency ref'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'Joule losses', 'Value': Pj_SL*Pj_ref, 'Unit': '[W]', 'Comment': 'Pj / Pj ref'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'Iron losses', 'Value': Pi_SL*Pi_ref, 'Unit': '[W]', 'Comment': 'Pi / P ref'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'Cooling power', 'Value': Pc_SL*Pc_ref, 'Unit': '[W]', 'Comment': 'Pc / Pc ref'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'Cooling system efficiency', 'Value': kc_SL, 'Unit': '[-]', 'Comment': 'kc / kc ref'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'Motor power', 'Value': Power/eff_SL, 'Unit': '[W]', 'Comment': 'Power corrected by the motor efficiency'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'Efficiency', 'Value': eff_SL, 'Unit': '[-]', 'Comment': 'Efficiency of the motor'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'Motor mass', 'Value': m_motor_SL*m_motor_ref, 'Unit': '[kg]', 'Comment': 'm_motor / m_motor ref'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'Gear mass', 'Value': m_gear, 'Unit': '[kg]', 'Comment': 'm_motor / m_motor ref'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'Gear reduction ratio', 'Value': M0, 'Unit': '[-]', 'Comment': 'm_motor / m_motor ref'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'Power electronic mass', 'Value': m_power_electronic, 'Unit': '[kg]', 'Comment': 'm_motor / m_motor ref'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'IGBT losses', 'Value': J_IGBT, 'Unit': '[W]', 'Comment': 'm_motor / m_motor ref'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'Diode losses', 'Value': J_diode, 'Unit': '[W]', 'Comment': 'm_motor / m_motor ref'}])[col_names]
        items = sorted(df['Type'].unique().tolist())
        def fun(Type):
            return df[df['Type']==Type] 
        widgets.interact(fun, Type=items)
        return fun


In [21]:
#Vector of parameterl
        #J_SL
param = [0.5,\
        #D/l_SL
        0.5,\
        #switching frequency
        9000,\
        #inductor: airgap/diam, current density
        10**(-1),10.5*10**6,\
        #conductor aspect ratio, oversizing coefficient
        0.6,1.5,\
        #IGBT oversizing coefficient, width heatsink 
        3.3,0.105]

#initial characteristics before optimization
print("-----------------------------------------------")
print("Initial characteristics before optimization :")
sizing_code(param, 'Prt')
print("-----------------------------------------------")

-----------------------------------------------
Initial characteristics before optimization :


C:\Users\Chiara\AppData\Local\Temp\ipykernel_26224\2631429099.py:215: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([{'Type': 'Motor', 'Name': 'Propeller power', 'Value': Power, 'Unit': '[W]', 'Comment': 'power'}])[col_names]
C:\Users\Chiara\AppData\Local\Temp\ipykernel_26224\2631429099.py:216: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([{'Type': 'Motor', 'Name': 'Propeller torque', 'Value': Torque, 'Unit': '[Nm]', 'Comment': 'Torque'}])[col_names]
C:\Users\Chiara\AppData\Local\Temp\ipykernel_26224\2631429099.py:217: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([{'Type': 'Motor', 'Name': 'Motor RPM', 'Value': RPM_new, 'Unit': '[tr/min]', 'Comment': 'RPM'}])[col_names]
C:\User

interactive(children=(Dropdown(description='Type', options=('Motor',), value='Motor'), Output()), _dom_classes…

-----------------------------------------------


In [27]:
objectif = lambda x: sizing_code(x, 'Obj')
# Optimization bounds

        # J_Sl_climb
bounds = [(0.1,1),\
         # d/l_SL
         (0.1,1),\
         #switching frequency
         (2000,30000),\
         #inductor: airgap/diam, current density
         (10**(-3),10**(-1)),(1*10**6,20*10**6),\
         #conductor aspect ratio, oversizing coefficient
         (0.3,0.8),(1,5),\
         #IGBT oversizing coefficient, width heatsink 
         (0.5,10),(0.04,0.12)]

 # SLSQP omptimisatio5
result = scipy.optimize.fmin_slsqp(func=objectif, x0=param, eqcons=[], bounds=bounds,iter=500, acc=1e-5,epsilon=1e-3)

#result = scipy.optimize.differential_evolution(func=objectif,bounds=bounds,tol=1e-12, maxiter=500, polish=bool)
#result = scipy.optimize.minimize(fun=objectif, x0=param, bounds=bounds)
#result = scipy.optimize.fmin(func=objectif, x0=param, maxiter=10**6)


# Final characteristics after optimization 
print("----------------------------------------------")
print("Final characteristics after optimization :")

print("-----------------------------------------------")
print('Current density Scaling_Law motor =',result[0])
print('D over L motor ratio =',result[1])
print('switching frequency =',result[2])
print('inductor: airgap/diam =',result[3])
print('inductor: current density =', result[4])
print('conductor aspect ratio =',result[5])
print('conductor oversizing coefficient =',result[6])
print('IGBT oversizing coefficient =', result[7])
print('IGBT width heatsink =',result[8])


sizing_code(result, 'Prt')

Optimization terminated successfully    (Exit mode 0)
            Current function value: 49.03255997022827
            Iterations: 2
            Function evaluations: 20
            Gradient evaluations: 2
----------------------------------------------
Final characteristics after optimization :
-----------------------------------------------
Current density Scaling_Law motor = 1.0
D over L motor ratio = 1.0
switching frequency = 9000.00017777775
inductor: airgap/diam = 0.1
inductor: current density = 10500000.0
conductor aspect ratio = 0.6
conductor oversizing coefficient = 1.0
IGBT oversizing coefficient = 3.3
IGBT width heatsink = 0.039999999999999994


C:\Users\Chiara\AppData\Local\Temp\ipykernel_26224\2631429099.py:215: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([{'Type': 'Motor', 'Name': 'Propeller power', 'Value': Power, 'Unit': '[W]', 'Comment': 'power'}])[col_names]
C:\Users\Chiara\AppData\Local\Temp\ipykernel_26224\2631429099.py:216: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([{'Type': 'Motor', 'Name': 'Propeller torque', 'Value': Torque, 'Unit': '[Nm]', 'Comment': 'Torque'}])[col_names]
C:\Users\Chiara\AppData\Local\Temp\ipykernel_26224\2631429099.py:217: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([{'Type': 'Motor', 'Name': 'Motor RPM', 'Value': RPM_new, 'Unit': '[tr/min]', 'Comment': 'RPM'}])[col_names]
C:\User

interactive(children=(Dropdown(description='Type', options=('Motor',), value='Motor'), Output()), _dom_classes…

<function __main__.sizing_code.<locals>.fun(Type)>